In [ ]:
!pip install --quiet gradio transformers

In [6]:
import gradio as gr
import sys
import io
import pandas as pd

class Logger:
    def __init__(self, filename):
        self.terminal = sys.stderr
        self.log = open(filename, "w")

    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)

    def flush(self):
        self.terminal.flush()
        self.log.flush()

    def isatty(self):
        return False

sys.stdout = Logger("output.log")

def read_logs():
      sys.stdout.flush()
      with open("output.log", "r") as f:
          return f.read()

def get_questions_list():
  path_to_data_files = "/content/drive/MyDrive/official/Creations/hackathon-GeminiAPIDeveloper - Aug2024/synthetic_dataset/csv_equivalents/"
  qa_file = path_to_data_files + "financial_literacy_quiz.csv"
  qa_df = pd.read_csv(qa_file)
  questions_list = []
  for row in qa_df.itertuples():
    question = row.Question
    options = [row.optionA, row.optionB, row.optionC, row.optionD]
    if row.Answer == "A":
      answer = row.optionA
    elif row.Answer == "B":
      answer = row.optionB
    elif row.Answer == "C":
      answer = row.optionC
    elif row.Answer == "D":
      answer = row.optionD

    questions_list.append({
              "question": question,
              "options": options,
              "answer": answer
          })
  return questions_list

def quiz():
    questions = get_questions_list()

    def check_answer(question_index, selected_option):
        if selected_option == questions[question_index]["answer"]:
            return f"Correct! The answer is {questions[question_index]['answer']}"
        else:
            return f"Incorrect. The correct answer is {questions[question_index]['answer']}"


    with gr.Blocks() as interface:

        console_output = gr.TextArea(label="Console Output", lines=10, max_lines=200) # Add this line

        question_index = gr.State(0)
        score = gr.State(0)

        gr.Markdown("## Multiple Choice Quiz")

        def display_question():
            return questions[question_index.value]["question"]

        def display_options():
            return questions[question_index.value]["options"]

        question_text = gr.Markdown(display_question())
        sys.stdout.write(f"\nquestion_text.value: {question_text.value}")

        option_buttons = gr.Radio(display_options(), label="Options")
        sys.stdout.write(f"\noption_buttons.value: {option_buttons.value}")

        check_button = gr.Button("Check Answer")
        sys.stdout.write(f"\ncheck_button.value: {check_button.value}")

        result_text = gr.Markdown("")
        sys.stdout.write(f"\nresult_text.value: {result_text.value}")

        score_text = gr.Markdown(f"Score: {score.value}")
        sys.stdout.write(f"\nscore_text.value: {score_text.value}")


        def update_question(selected_option):
            #nonlocal score, result_text, question_text, option_buttons, check_button, score_text

            sys.stdout.write(f"Selected option index: {selected_option}")  # Print the selected option index
            sys.stdout.write(f"\nCurrent question index: {question_index.value}")  # Print the current question index

            result = check_answer(question_index.value, selected_option)
            sys.stdout.write(f"\nResult from check_answer: {result}")  # Print the result of answer checking

            result_text.value = result
            sys.stdout.write(f"\nresult_text.value: {result_text.value}")
            if "Correct" in result:
                score.value += 1
            score_text.value = f"Score: {score.value}"
            question_index.value += 1

            sys.stdout.write(f"\nUpdated question index: {question_index.value}")  # Print the updated question index
            sys.stdout.write(f"\nscore_text.value: {score_text.value}")

            if question_index.value < len(questions):
                #question_text.value = display_question(),
                #option_buttons.choices = display_options()
              return(
                result,
                #display_question(),
                questions[question_index.value]["question"],
                #display_options(),
                #questions[question_index.value]["options"],
                gr.update(choices=questions[question_index.value]["options"]),  # Update choices here
                check_button.value,
                score_text.value
              )
            else:
              question_index.value = 0
              return(
                  result,
                  questions[question_index.value]["question"],
                  gr.update(choices=questions[question_index.value]["options"]),  # Update choices here
                  check_button.value,
                  score_text.value
              )

        check_button.click(update_question, inputs=[option_buttons], outputs=[result_text, question_text, option_buttons, check_button, score_text])
        interface.load(read_logs, None, console_output, every=1)

    interface.launch(show_error=True)

In [7]:
quiz()


question_text.value: What's the first step to creating a budget?
option_buttons.value: None
check_button.value: Check Answer
result_text.value: 
score_text.value: Score: 0Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://cfe6beecffc135d57a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
